# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Derek Fintel

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

#### Initial version check

In [32]:
import spacy
import spacytextblob

print(spacy.__version__)
print(spacytextblob.__version__)

3.8.7
5.0.0


1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [34]:
import requests
import json

result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

with open('lyrics.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotation?  Answer this question in a comment in your code cell.

In [48]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load & assign spaCy language model
nlp = spacy.load("en_core_web_sm")

# Add spacytextblob component by name (registers extensions on Doc)
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")

# Verify component is in the pipeline
print("Pipeline components:", nlp.pipe_names)

# Read lyrics from file
with open("lyrics.json", "r", encoding="utf-8") as f:
    data = json.load(f)
lyrics = data["lyrics"]

print("Lyrics preview:\n", lyrics[:100], "...")

# Process lyrics through the pipeline
doc = nlp(lyrics)

# Now the polarity extension attribute should be available
#print("\nPolarity Score:", doc._.polarity)

truncated_polarity = int(doc._.polarity * 100) / 100
truncated_subjectivity = int(doc._.subjectivity * 100) / 100

print("\nPolarity Score:", truncated_polarity)
print("Subjectivity Score:", truncated_subjectivity)



Pipeline components: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'spacytextblob']
Lyrics preview:
 I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not  ...

Polarity Score: 0.04
Subjectivity Score: 0.55


In [49]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from spacy.tokens import Doc
from textblob import TextBlob

# --- Manual registration of extension attributes ---

if not Doc.has_extension("blob"):
    Doc.set_extension("blob", getter=lambda doc: TextBlob(doc.text))

if not Doc.has_extension("polarity"):
    Doc.set_extension("polarity", getter=lambda doc: doc._.blob.polarity)

if not Doc.has_extension("subjectivity"):
    Doc.set_extension("subjectivity", getter=lambda doc: doc._.blob.subjectivity)

# --- Load spaCy model and add spacytextblob pipe ---

nlp = spacy.load("en_core_web_sm")

if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")

# --- Load lyrics JSON ---

with open("lyrics.json", "r", encoding="utf-8") as f:
    lyrics_data = json.load(f)

lyrics = lyrics_data.get("lyrics", "")
print("Lyrics preview:\n", lyrics[:500], "...\n")

# --- Process lyrics and analyze sentiment ---

doc = nlp(lyrics)

print(f"Polarity Score: {doc._.polarity}")
print(f"Subjectivity Score: {doc._.subjectivity}")


Lyrics preview:
 I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I have a secret to tell 

From my electrical well 

It's a simple message and I'm leaving out the whistles and bells 

So the room must liste ...

Polarity Score: 0.04505208333333333
Subjectivity Score: 0.5532738095238096


3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [47]:
import requests
import json
import os

def save_lyrics_to_file(artist, song, filename):
    """Fetch lyrics from lyrics.ovh API and save to a JSON file."""
    base_url = "https://api.lyrics.ovh/v1"
    url = f"{base_url}/{artist}/{song}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        lyrics_data = response.json()
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(lyrics_data, f, ensure_ascii=False, indent=4)
        print(f"✅ Saved lyrics for '{song}' by {artist} to '{filename}'")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching lyrics for '{song}' by {artist}: {e}")

# ✅ Test with 4 songs
save_lyrics_to_file("Coldplay", "Yellow", "yellow_lyrics.json")
save_lyrics_to_file("Adele", "Hello", "hello_lyrics.json")
save_lyrics_to_file("The Beatles", "Hey Jude", "hey_jude_lyrics.json")
save_lyrics_to_file("Radiohead", "Creep", "creep_lyrics.json")


❌ Error fetching lyrics for 'Yellow' by Coldplay: 504 Server Error: Gateway Time-out for url: https://api.lyrics.ovh/v1/Coldplay/Yellow
✅ Saved lyrics for 'Hello' by Adele to 'hello_lyrics.json'
✅ Saved lyrics for 'Hey Jude' by The Beatles to 'hey_jude_lyrics.json'
✅ Saved lyrics for 'Creep' by Radiohead to 'creep_lyrics.json'


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [50]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy with spacytextblob (if not already loaded)
nlp = spacy.load("en_core_web_sm")
if 'spacytextblob' not in nlp.pipe_names:
    nlp.add_pipe('spacytextblob')

def get_lyrics_polarity(filename):
    """Load lyrics from file and return sentiment polarity score."""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
        lyrics = data.get("lyrics", "")
        doc = nlp(lyrics)
        return doc._.polarity

# ✅ Evaluate three songs and print results
songs = {
    "Yellow - Coldplay": "yellow_lyrics.json",
    "Hello - Adele": "hello_lyrics.json",
    "Creep - Radiohead": "creep_lyrics.json"
}

print("🎵 Polarity Scores:")
for title, file in songs.items():
    polarity = get_lyrics_polarity(file)
    print(f"{title}: Polarity = {polarity:.3f}")

# 📝 Interpretation:
# Based on the polarity scores printed above,
# you can compare the sentiment with your subjective interpretation.


🎵 Polarity Scores:
Yellow - Coldplay: Polarity = 0.243
Hello - Adele: Polarity = -0.141
Creep - Radiohead: Polarity = 0.579
